# Playing with data

read data from csv

In [91]:
import pandas as pd
df_season = pd.read_csv("static/data/NBA_season.csv")
df_season = df_season[df_season.columns.tolist()[1:]]
df_playoffs = pd.read_csv("static/data/NBA_playoffs.csv")
df_playoffs= df_playoffs[df_playoffs.columns.tolist()[1:]]

## Calculate average stats by pos

In [9]:
group = df_season[(df_season['Year'] >= 1980) & (df_season['MP'] >= 20)].groupby('Pos')

df_avg_stats_by_pos_mixed = group[["AST", "PTS", "TOV", "BLK", "STL", "ORB", "DRB"]].mean()
df_avg_stats_by_pos = df_avg_stats_by_pos_mixed.loc[['PF', 'C', 'PG', 'SF', 'SG']]
df_count_by_pos = group['id'].count()

In [10]:
count_by_pos = {
    'C': df_count_by_pos.loc['C'],
    'PF': df_count_by_pos.loc['PF'],
    'PG': df_count_by_pos.loc['PG'],
    'SF': df_count_by_pos.loc['SF'],
    'SG': df_count_by_pos.loc['SG'],
}

for pos, row in df_avg_stats_by_pos_mixed.iterrows():
    if pos not in ['PF', 'C', 'PG', 'SF', 'SG']:
        # get amount of players with this mixed position
        amount_mixed = df_count_by_pos.loc[pos]

        # get the sum of the mixed stats for each stat
        stats = []
        for stat in row:
            stats.append(stat * amount_mixed)

        # for each position, add avg to position
        for p in pos.split(','):
            p = p.strip()
            amount_pure = count_by_pos[p]

            # get average stats for this pure pos
            pos_row = df_avg_stats_by_pos.loc[p]
            for i in range(len(stats)):
                # create new weighted arithmetic mean
                new_stat = (pos_row[i] * amount_pure + stats[i]) / (amount_pure + amount_mixed)
                pos_row[i] = new_stat

            # update amount for this pos
            count_by_pos[p] += amount_mixed

In [11]:
df_avg_stats_by_pos

,AST,PTS,TOV,BLK,STL,ORB,DRB
Pos,,,,,,,
PF,1.949051,13.090512,1.768501,0.792694,0.848292,2.149146,5.082638
C,1.637459,12.275679,1.771553,1.434745,0.707383,2.469707,5.542997
PG,5.697905,12.883524,2.184571,0.198571,1.284857,0.674095,2.477524
SF,2.516463,14.219003,1.799059,0.504610,1.034807,1.398589,3.579210
SG,3.156130,14.305517,1.815937,0.308581,1.099650,0.859370,2.641856


## Calculate player score for season and playoffs

option 1 - best year of the player

In [93]:
def get_best_year(df):
    ser = df.groupby(['id'])['score2'].max()
    d = {}
    for key, value in ser.iteritems():
        entry = df[(df['id'] == key) & (df['score2'] == value)].iloc[0]
        d[key] = [entry['Name'], entry['Year'], entry['score2']]

    return pd.DataFrame.from_dict(d, orient="index", columns=['Name', 'Year', 'score2']).sort_values(by="score2", ascending=False)

option 2 - average of last max 4 years

In [94]:
def get_best_4_years(df):
    rows = {}

    for player in df['id'].unique():
        df_player = df[df['id'] == player].sort_values(by='score2', ascending=False).head(4)
        avg = df_player['score2'].mean()

        rows[player] = [df_player.iloc[0]['Name'], list(df_player['Year'].sort_values()), avg]

    best4 = pd.DataFrame().from_dict(rows, orient='index', columns=['Name', 'Years', 'score2']).sort_values(by='score2', ascending=False)
    best4.index.name = 'id'
    return best4

In [97]:
get_best_4_years(df_playoffs).loc['jordami01']

Name                Michael Jordan
Years     [1986, 1989, 1990, 1991]
score2                   31.431289
Name: jordami01, dtype: object

In [114]:
import numpy as np
best4_playoffs = get_best_4_years(df_playoffs)
def concat_avg_to_4years_playoff(row):
    
    print("before")
    if row.name in best4_playoffs.index:
        avg = np.mean([best4_playoffs.loc[row.name]['score2'], row['score2']])
    else:
        avg = row['score2']/2
    print("after")
    return avg

con = get_best_year(df_season)
con['avg'] = con.apply(concat_avg_to_4years_playoff, axis=1)

before
after
before
after


KeyboardInterrupt: 

In [ ]:
con

In [45]:
df_best4_playoffs.sort_values(by='score2', ascending=False)

,Name,Years,score2
id,,,
jordami01,Michael Jordan,"[1986, 1989, 1990, 1991]",31.431289
jamesle01,LeBron James,"[2009, 2017, 2018, 2020]",30.090656
johnsma02,Magic Johnson,"[1985, 1986, 1987, 1990]",29.743493
doncilu01,Luka Dončić,"[2020, 2021]",28.411557
stockjo01,John Stockton,"[1988, 1989, 1990, 1991]",28.329942
...,...,...,...
ntilila01,Frank Ntilikina,[2021],0.538628
higgise01,Sean Higgins,[1991],0.538628
newbiiv01,Ivano Newbill,[1997],0.484641
